##GPT-2를 이용한 문장 생성
사전 학습된 한국어 GPT‐2 를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라
앞으로 사전 학습된 GPT 를 사용할 때는 transformers 라는 패키지를 자주 사용하게 됩니다. 실습 환경에
transformers 패키지를 설치해둡시다.

In [1]:
!pip install transformers

###1. KoGPT-2로 문장 생성하기
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 와 마찬가지로 GPT 는 이미 누
군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야
합니다.

In [2]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel

TFGPT2LMHeadModel.from_pretrained(‘GPT 모델 이름’) 을 넣으면 두 개의 문장이 이어지는 문장 관계
인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면
해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [3]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

tokenizer.json: 0.00B [00:00, ?B/s]

GPT 가 생성할 문장의 방향성을 알려주기 위해서 시작 문자열을 정해줍니다. 여기서는 ‘근육이 커지기 위
해서는’ 이라는 문자열을 주고 GPT 에게 이어서 문장을 생성해보라고 해봅시다.

In [4]:
sent = '근육이 커지기 위해서는'

GPT 의 입력으로는 정수 인코딩 된 결과가 입력되어야 하므로 tokenizer.encode() 를 통해서’ 근육이 커지
기 위해서는’ 이라는 문자열을 정수 시퀀스로 변환해줍니다.

In [5]:
input_ids = tokenizer.encode(sent)

# tf.convert_to_tensor() 함수는 리스트, 튜플, 넘파이 배열, 스칼라 값 등 거의 모든 Python 객체를 Tensor로 바꿈
input_ids = tf.convert_to_tensor([input_ids])

print(input_ids)

tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


33245 10114 12748 11357 라는 5 개의 정수 시퀀스를 얻습니다. 해당 정수 시퀀스를 GPT 의 입력으로 사
용하여 GPT 가 이어서 문장을 생성하도록 해봅시다. 주어진 문장으로부터 이어서 문장을 생성하도록 하
는 것은 model.generate() 를 사용합니다.

In [6]:
output = model.generate(input_ids, max_length=128, repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


기존의 33245 10114 12748 11357 라는 5 개의 정수 시퀀스 뒤에도 여러 정수들이 추가로 생성된 것을 볼
수 있습니다. 정수들이 단순히 나열된 것만으로는 GPT 가 실제로 어떤 문장을 생성했는지 알기 어려우니
해당 정수 시퀀스를 한국어로 변환해봅시다. 이 과정은 tokenizer.decode() 를 사용하여 가능합니다.

In [7]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

‘근육이 커지기 위해서는’ 라는 문자열에 이어서 그 뒤에 근육이 커지기 위한이라는 문맥에 맞는듯한 글들
을 생성합니다. 물론 GPT 가 생성한 문장들은 문맥상 그럴듯해 보이지만 실제 사실 여부와는 다를 수 있
으니 이 점은 늘 주의해야 합니다.

###2. Numpy로 Top5 뽑기
GPT 는 기본적으로 이전 단어들로부터 다음 단어를 예측하는 언어 모델 (Language Model) 입니다. 위의
실습에서 확인한 바와 같이 ‘근육이 커지기 위해서는’ 이라는 입력을 넣었을 때 GPT 는 다음 단어로 ‘무엇
보다’ 라는 단어를 예측했었는데요. 실제로는 수많은 후보의 다음 단어들이 있었지만, 그 중 가장 확률이
높은 단어. 즉, Top 1 의 단어인’ 무엇보다’ 를 예측한 것입니다. 그렇다면 다음 단어로 또 어떤 후보들이
있었는지 Top 5 의 단어들을 뽑아봅시다. model() 에다가’ 근육이 커지기 위해서는’ 의 정수 인코딩 된 결
과를 입력으로 넣은 후 가장 확률이 높은 Top 5 의 단어들을 뽑아냅니다.

In [8]:
output = model(input_ids)
top5 = tf.math.top_k(output.logits[0, -1], k=5)

In [9]:
# 그 후에 TOP5의 단어를 한국어로 변환하여 출력해봅시다.
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['▁무엇보다', '▁우선', '▁반드시', '▁피부', '▁무엇보다도']

‘무엇보다’ 라는 단어 외에도 ‘우선’, ‘반드시’, ‘피부’, ‘무엇보다도’ 라는 4 개의 단어가 높은 확률로 선택될
수 있었음을 보여줍니다.

###3. Numpy Top5로 문장 생성하기
앞서 문장을 생성했을 당시에는 각 시점(time step)마다 가장 확률이 높은 단어를 예측했지만, 이번에는 매 시점마다 Top5개의 단어들 중에서 랜덤으로 선택하는 방식을 택하여 문장을 생성해 봅시다.

In [10]:
sent ='근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:
  output = model(np.array(input_ids))

  # Top5의 단어들을 추출
  top5 = tf.math.top_k(output.logits[0,-1], k=5)

  # Top5의 단어들 중 랜덤으로 다음 단어 선택
  token_id = random.choice(top5.indices.numpy())
  input_ids.append(token_id)

tokenizer.decode(input_ids)

'근육이 커지기 위해서는 피부 속 콜라젠의 활성산소 농도를 높이는 것도 도움이 될 것이다.\n특히 요즘은 미세 먼지와 황사가 심해져 피부 트러스가 심해지는 계절이라, 미세 입자가 많은 화장품 사용을 줄이고 싶지만 피부 깊숙이 침투하는 활성산'

## GPT-2를 이용한 한국어 챗봇


###1. KoGPT-2의 모델과 토크나이저 로드
transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT 와 마찬가지로 GPT 는 이미 누
군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야
합니다.

In [11]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

TFGPT2LMHeadModel.from_pretrained(‘GPT 모델 이름’) 을 넣으면 두 개의 문장이 이어지는 문장 관계
인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained(‘모델 이름’) 을 넣으면
해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다. 토크나이저를 로드할 때 텍스
트 생성을 시작할 때 사용하는 시작 토큰을 bos_token, 텍스트 생성을 종료할 때 사용하는 종료 토큰을
eos_token 이라는 이름으로 지정할 수 있습니다. 여기서는 <</s>/s>라는 토큰을 시작 토큰이자 종료 토큰으
로 사용하고자 합니다.

In [12]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>',
                                          eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

GPT에는 특별한 용도로 사용하기 위해서 할당해놓은 특별 토큰(special token)들이 존재합니다. Ko-GPT2에서 사용되는 특별 토큰들의 정수 그리고 반대로 정수 1,2,3,4는 각각 어떤 토큰들을 의미하는지 확인해봅시다.

In [13]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-'*25)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
-------------------------
</s>
<usr>
<pad>
<sys>


주로 문장의 시작을 의미하는 용도로 사용되는 BOS 토큰과 주로 문장의 종료를 의미하는 용도로 사용되는
EOS token 이 정수로는 몇 번인지 확인하기 위해서 tokenizer.bos_token_id 와 tokenizer.eos_token_id
를 각각 출력했습니다. 또한 패딩 토큰이 정수로는 몇 번인지 확인하기 위해서 tokenizer.pad_token_id
를 출력했습니다. 확인 결과, KoGPT‐2 에서는 BOS 토큰과 EOS 토큰이 동일한 정수 1 로 맵핑되는 것으로
확인되었으며 (이것은 KoGPT‐2 제작자들이 이렇게 정한 것입니다.) 패딩 토큰은 정수 3 으로 맵핑되는 것
을 확인했습니다.

반대로 정수 1, 2, 3, 4 를 단어로 표현할 경우를 출력해보았습니다. 정수 1. 즉, BOS 토큰이기도 하고, EOS
토큰이기도 한 토큰은 </s>로, 패딩 토큰인 정수 3 은 <pad>로 출력되는 것을 확인했습니다. 정수 2 와
정수 4 의 경우에는 <usr>와 <sys>라는 토큰으로 맵핑되어져 있었는데, 이는 GPT 로 대화 모델을 만들
경우에 각각 유저 (User) 와 시스템 (System) 을 구분하기 위한 용도입니다. 이번 실습에서도 실제로 이 두
토큰을 이용하여 챗봇을 구현할 것이므로 어떠한 의미인지는 뒤에서 실습을 통해 이해해보겠습니다.

### 2. 챗봇 데이터 로드


In [14]:
import pandas as pd
import tqdm
import urllib.request

In [15]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
print('챗 봇 데 이 터 의 개 수 :', len(train_data))

챗 봇 데 이 터 의 개 수 : 11823


챗봇 데이터의 개수는 11,823 개로 해당 데이터는 앞서 **'BERT 의 문장 임베딩 (SBERT) 을 이용한 한국어 챗봇'** 에서 사용한 데이터와 동일하므로 데이
터에 대한 설명은 생략합니다.

###3. 챗봇 데이터 전처리
챗봇 데이터를 전처리하기 위한 함수 get_chat_data()를 만듭니다.

In [16]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)

    yield bos_token + sent + eos_token
  # yield는 값을 하나씩“잠깐 내보내고”함수 실행을 멈춘 뒤, 다음 호출 때 다시 이어서 실행할 수 있게 해줘요.


위의 전처리는 <usr<usr>> 다음에 사용자의 질문을 부착하고, 그 후 <sys<sys>> 다음에 챗봇의 답변을 부착하는
형식으로 전처리를 진행합니다. 그리고 앞 뒤에 시작 토큰 (BOS 토큰) 과 종료 토큰 (EOS 토큰) 을 부착하
는 전처리입니다. 앞에서 확인했듯이 두 토큰은 동일하며 </s</s>>입니다. 전처리 후에 데이터가 어떻게 변
경되는지는 뒤에서 확인합니다. 배치 크기를 32 로 하고, 패딩 토큰으로 패딩을 진행합니다. 위에서 확인
했듯이 패딩 토큰의 정수는 3 입니다.

In [17]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32, output_shapes=(None,))

# 배치 크기 만큼 데이터를 구성하되 패딩 토큰으로 패딩을 진행.
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,),
                               padding_values=tokenizer.pad_token_id)

첫번째 배치 (첫 32 개의 데이터 묶음) 을 출력해봅시다.

In [18]:
# 첫 배치 출력
for batch in dataset:
  print(batch)
  break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3     3     3     3
      3     3     3     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  800

각 데이터의 맨 앞에는 정수 1, 2 가 부착되어져 있습니다. 정수 1, 2 는 앞에서
확인했던 것과 같이 각각 </s>와 <user>에 해당됩니다. 첫번째 배치는 현재 batch 라는 변수에 저장되
어져 있으므로 32 개의 데이터 중 가장 첫번째 샘플을 출력해봅시다. 이때 이미 정수 인코딩이 진행된 상
황이므로 tokenizer.decode() 를 통해 정수 인코딩 된 결과를 다시 복원하여 출력합니다.

In [19]:
# 첫 번 째 배 치(32개 의 샘 플) 중 첫 번 째 샘 플 출 력
print(tokenizer.decode(batch[0]))

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


위의 get_chat_data() 함수로 전처리를 진행할 당시의 의도대로 사용자의 질문 앞에 <usr>가 붙어있고,
챗봇의 답변 앞에는 <sys>가 붙어있습니다. 그리고 사용자의 질문과 챗봇의 답변 앞 뒤로는 시작 토큰
과 종료 토큰에 해당하는 </s>이 붙어있으며, 그 뒤에는 패딩 토큰인 <pad>가 붙습니다. 이제 학습을
진행해봅시다.

###4. 챗봇 학습하기


In [20]:
# 옵티마이저 결정
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 전체 데이터의 개수를 배치 크기로 나누면 하나의 에포크에서 실행되는 학습 횟수가 계산됨
steps = len(train_data) // batch_size + 1

EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
    with tf.GradientTape() as tape:
      result = model(batch, labels=batch)
      loss = result[0]
      batch_loss = tf.reduce_mean(loss)

    grads = tape.gradient(batch_loss, model.trainable_variables)
    adam.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss += batch_loss / steps
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.12697864


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.69872737


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.37745833


###5. 챗봇 실행하기
이제 챗봇에게 답변을 얻기 위해서 어떤 전처리를 진행해야 할 지 차근차근 정리해봅시다. 우선 앞서 모
델 학습 전에 전처리를 진행했던 것처럼 사용자의 질문의 앞 뒤에 <usr<usr>>와 <sys<sys>>를 부착합니다. 그 후
해당 문장 앞에 시작 토큰에 해당하는 </s</s>>를 부착합니다.

In [21]:
text = '오늘도 좋은 하루!'
sent = '<usr>' + text + '<sys>'
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print('정 수 인 코 딩 후 :', input_ids)
print('정 수 인 코 딩 을 재 복 원 :', tokenizer.decode(input_ids[0]))

정 수 인 코 딩 후 : tf.Tensor([[    1     2 10070  7235 10586 12557   376     4]], shape=(1, 8), dtype=int32)
정 수 인 코 딩 을 재 복 원 : </s><usr> 오늘도 좋은 하루!<sys>


현재의 입력은 챗봇이 학습할 당시의 형태 (</s</s>><usr<usr>>사용자의 질문<sys<sys>>챗봇의 답변</s</s>>) 에서
챗봇의 답변이 바로 시작되기 전까지의 형태입니다. 기존에 학습하였을 당시에는 <sys<sys>> 뒤에 챗봇의 답변이 있었으므로 현재의 입력을 KoGPT‐2 에 입력으로 사용하면 챗봇은 학습때 봤던 데이터의 형식 그대
로 챗봇의 답변을 작성하려는 관성을 갖고 있습니다. 현재의 입력을 모델의 입력으로 넣고 모델이 생성하
는 문장을 얻습니다.

In [22]:
output = model.generate(input_ids, max_length=50, early_stopping=True,
                        eos_token_id=tokenizer.eos_token_id)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


</s><usr> 오늘도 좋은 하루!<sys> 좋은 하루네요.</s>


KoGPT‐2 의 출력은 사용자가 넣어준 입력이었던 </s</s>><usr<usr>> 오늘도 좋은 하루!<sys<sys>>도 포함하여
반환되므로 챗봇의 답변만 확인하기 위해서는 <sys<sys>>를 기준으로 분할하여 뒷 부분만 가져와야 합니
다.

In [23]:
print(decoded_sentence.split('<sys> ')[1].replace('</s>',''))

좋은 하루네요.


만약 동일한 질문에도 KoGPT‐2 의 답변이 랜덤성을 갖기를 원한다면 do_sample=True, top_k=10 이라
는 파라미터를 통해서 후보가 되는 단어 10 개 중 랜덤으로 선택하여 생성하도록 유도할 수 있습니다.

In [31]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))

오늘도 좋은 하루!


In [38]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id]+ tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [39]:
return_answer_by_chatbot('안녕! 반가워~')

'반가워요.'

In [40]:
return_answer_by_chatbot('너는 누구야?')

'저는 짝사랑하는 사람입니다.'

In [41]:
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

'짝사랑은 영원할 거예요.'

In [42]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

'영화 추천해달라고 해보세요.'

In [43]:
return_answer_by_chatbot('너 딥러닝 잘해?')

'잘할 수 있을 거예요.'

KoGPT‐2 는 상대적으로 모델의 크기가 작습니다. 챗봇의 성능을 올리고 싶다면 데이터의 양을 증량하거
나 T5‐Large 와 같은 보다 큰 모델을 시도하시기를 권합니다.